In [ ]:
import pandas as pd

In [ ]:
import tensorflow as tf
# Getting GPU device name.
device_name = tf.test.gpu_device_name()

if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
import torch
# If a GPU is available
if torch.cuda.is_available():    
    #set device to GPU   
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If no GPU is available
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install transformers


import numpy          as np


import transformers
from transformers                     import  RobertaModel, RobertaTokenizer, AdamW, get_linear_schedule_with_warmup
import torch



from sklearn.metrics                  import classification_report


from torch                            import nn, optim
from torch.utils                      import data


RANDOM_SEED = 64
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

if torch.cuda.is_available():
   torch.cuda.manual_seed(RANDOM_SEED)
   torch.cuda.manual_seed_all(RANDOM_SEED) 
   torch.backends.cudnn.deterministic = True  
   torch.backends.cudnn.benchmark = False


CLASS_NAMES =  ['true','false','unverified']
MAX_LENGTH = 200
BATCH_SIZE = 16
EPOCHS = 6
HIDDEN_UNITS = 128

tokenizer = transformers.RobertaTokenizer.from_pretrained('roberta-large')

In [ ]:
trainDf = pd.read_csv('/content/drive/MyDrive/NLP/csvfiles/taskb/trainingData.csv')
trainDf

,text,id,label,labelvalue
0,France: 10 people dead after shooting at HQ of...,552783667052167168,true,0
1,BREAKING: 10 reportedly shot dead at Paris HQ ...,552785375161499649,true,0
2,BREAKING: At least 10 killed in shooting at Fr...,552791196247269378,true,0
3,Eleven dead in shooting at Paris offices of sa...,552791578893619200,true,0
4,BREAKING Charlie Hebdo latest: 11 dead 10 woun...,552792544132997121,true,0
...,...,...,...,...
321,[Serious] Is it true that 85% of people can on...,8gp0d7,true,0
322,Did Eric Schneiderman Help NXIVM Sell Child Se...,8i274x,false,1
323,Debunk this: The direction you sleep matters,8n9173,false,1
324,Is it true police officers must hit a quota at...,8unvgg,false,1


In [ ]:
trainDf['label'].isna().sum()

0

In [ ]:
devDf = pd.read_csv('/content/drive/MyDrive/NLP/csvfiles/taskb/testData.csv')
devDf

,text,id,label,labelvalue
0,#Breaking: Pentagon releases video of the “mot...,852844529778601985,false,1
1,Rep. Sheila Jackson Lee has no shame. I still ...,443938194715713536,true,0
2,"ICYMI: ""Rep Sheila Jackson Lee (D-Tx) Wants Hu...",774165935041093633,true,0
3,"In 2003, Houston U.S. Rep. Sheila Jackson Lee ...",909084965035487232,true,0
4,Noah’s Ark Theme Park Destroyed in a Flood? $URL$,897502435593265152,false,1
...,...,...,...,...
76,People who can't smell. Is it true that there ...,8m9y0o,true,0
77,Great Pyramid of Giza can focus electromagneti...,938o5h,true,0
78,Is it true that the federal government require...,ayn70,false,1
79,Is it true Romney has Caymen bank accounts?,wfajw,true,0


In [ ]:
class StanceDataset(data.Dataset):

  def __init__(self, firstSeq, TextSrcInre, labelValue,  tokenizer, max_len):
    self.firstSeq    = firstSeq         
    self.labelValue  = labelValue   
    self.tokenizer   = tokenizer     
    self.max_len     = max_len       

  def __len__(self):
    return len(self.labelValue)

  def __getitem__(self, item):
    firstSeq    = str(self.firstSeq[item])

    
    encoding = tokenizer.encode_plus(
        firstSeq,
        max_length = self.max_len,
        add_special_tokens= True,
        truncation = True,
        pad_to_max_length = True,
        return_attention_mask = True,
        return_tensors = 'pt'
    )

    return {
        'firstSeq' : firstSeq,
        'input_ids': encoding['input_ids'].flatten(),
        'attention_mask': encoding['attention_mask'].flatten(),
        'labelValue'  : torch.tensor(self.labelValue[item], dtype=torch.long)
    }

In [ ]:

def createDataLoader(dataframe, tokenizer, max_len, batch_size):
  ds = StanceDataset(
      firstSeq    = dataframe.text.to_numpy(),
      labelValue  = dataframe.labelvalue.to_numpy(),
      tokenizer   = tokenizer,
      max_len     = max_len
  )

  return data.DataLoader(
      ds,
      batch_size  = batch_size,
      shuffle     = True,
      num_workers = 3
  )


In [ ]:

trainDataLoader        = createDataLoader(trainDf, tokenizer, MAX_LENGTH, BATCH_SIZE)

developmentDataLoader  = createDataLoader(devDf, tokenizer, MAX_LENGTH, BATCH_SIZE)


In [ ]:
class StanceClassifier(nn.Module):

  def __init__(self,  n_classes):
    super(StanceClassifier, self).__init__()
    self.robertaModel              = RobertaModel.from_pretrained('roberta-large')    
                                    

    self.drop                      = nn.Dropout(p = 0.3)

    self.output                    = nn.Linear(self.robertaModel.config.hidden_size, n_classes)
    
    self.softmax                   = nn.Softmax(dim = 1)

  def forward(self, input_ids, attention_mask):
    
    roberta_output     = self.robertaModel(
        input_ids      = input_ids,               
        attention_mask = attention_mask )        

        pooled_output = roberta_output[1]          
        output        = self.drop(pooled_output)
        output        = self.output(output)
    
    return self.softmax(output)

In [ ]:
model = StanceClassifier(len(CLASS_NAMES))
model = model.to(device)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
optimizer = AdamW(model.parameters(), 
                  lr = 2e-6, 
                  correct_bias= False)

totalSteps = len(trainDataLoader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=0,
            num_training_steps = totalSteps
)



weights      = [0.75, 1.37, 1.04]
classWeights = torch.FloatTensor(weights)
lossFunction = nn.CrossEntropyLoss(weight = classWeights).to(device)

In [ ]:
def train_epoch(
  model,
  dataLoader,
  lossFunction,
  optimizer,
  device,
  scheduler,
  n_examples
):

  model = model.train()
  losses = []
  correctPredictions = 0

  for d in dataLoader:
    
    input_ids              = d["input_ids"].to(device)                          
    attention_mask         = d["attention_mask"].to(device)                     
    labelValues            = d["labelValue"].to(device)                          

    outputs = model(
      input_ids             = input_ids,
      attention_mask        = attention_mask,
    )

    _, predictionIndices = torch.max(outputs, dim=1)
    loss = lossFunction(outputs, labelValues)
  
    correctPredictions += torch.sum(predictionIndices == labelValues)
    losses.append(loss.item())
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

  return np.mean(losses), correctPredictions.double() / n_examples

In [ ]:
def eval_model(
    model, 
    dataLoader, 
    lossFunction,
    device,
    n_examples
    ):
  
  model = model.eval()
  losses = []
  correctPredictions = 0

  with torch.no_grad():
    for d in dataLoader:

      attention_mask         = d["attention_mask"].to(device)                     
      labelValues            = d["labelValue"].to(device)                         


      outputs = model(
        input_ids             = input_ids,
        attention_mask        = attention_mask,
  
       
      )

      _, predictionIndices = torch.max(outputs, dim=1)
      loss = lossFunction(outputs, labelValues)

      correctPredictions += torch.sum(predictionIndices == labelValues)
      losses.append(loss.item())

  return np.mean(losses), correctPredictions.double() / n_examples

In [ ]:
for epoch in range(EPOCHS):
  print(f'Epoch {epoch + 1}')
  trainLoss, trainAccuracy = train_epoch(
    model,
    trainDataLoader,
    lossFunction,
    optimizer,
    device,
    scheduler,
    len(trainDf)
  )
  
  print(f'Training loss {trainLoss} Training accuracy {trainAccuracy}')

  devLoss, devAccuracy = eval_model(
    model,
    developmentDataLoader,
    lossFunction,
    device,
    len(devDf)
  )

  print(f'Development loss {devLoss} Development accuracy {devAccuracy}')
  print()
  
  print()

Epoch 1
Training loss 1.1061964772996449 Training accuracy 0.2668711656441718
Development loss 1.0991637706756592 Development accuracy 0.49382716049382713


Epoch 2
Training loss 1.0985755977176486 Training accuracy 0.2852760736196319
Development loss 1.0803916255633037 Development accuracy 0.4567901234567901


Epoch 3
Training loss 1.0705555166516985 Training accuracy 0.4447852760736196
Development loss 1.0949122508366902 Development accuracy 0.37037037037037035


Epoch 4
Training loss 1.0404798956144423 Training accuracy 0.5521472392638037
Development loss 1.1730943123499553 Development accuracy 0.25925925925925924


Epoch 5
Training loss 0.9907456636428833 Training accuracy 0.6503067484662577
Development loss 1.111816147963206 Development accuracy 0.37037037037037035


Epoch 6
Training loss 0.9860430331457228 Training accuracy 0.647239263803681
Development loss 1.0752997001012166 Development accuracy 0.4074074074074074




In [ ]:
def get_predictions(model, data_loader):

  model = model.eval()
  review_texta = []
  review_textb = []
  predictions = []
  prediction_probs = []
  real_values = []

  with torch.no_grad():
    for d in data_loader:

      textas                 = d["firstSeq"]
      input_ids              = d["input_ids"].to(device)
      attention_mask         = d["attention_mask"].to(device)
      labels                 = d["labelValue"].to(device)
    
      outputs = model(
        input_ids             = input_ids,
        attention_mask        = attention_mask,
       
      )

      _, preds = torch.max(outputs, dim=1)    

      review_texta.extend(textas)
      predictions.extend(preds)
      prediction_probs.extend(outputs)
      real_values.extend(labels)

  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  real_values = torch.stack(real_values).cpu()
  
  return review_texta, predictions, prediction_probs, real_values

In [ ]:
firstSeq_dev, yHat_dev, predProbs_dev, yTest_dev = get_predictions(
  model,
  developmentDataLoader
)

In [ ]:
print(classification_report(yTest_dev, yHat_dev, target_names= CLASS_NAMES))

              precision    recall  f1-score   support

        true       0.33      0.13      0.19        31
       false       0.47      0.70      0.56        40
  unverified       0.11      0.10      0.11        10

    accuracy                           0.41        81
   macro avg       0.30      0.31      0.28        81
weighted avg       0.37      0.41      0.36        81



In [ ]:
predProbs_dev

tensor([[0.2714, 0.4338, 0.2949],
        [0.2311, 0.4465, 0.3223],
        [0.2498, 0.4104, 0.3398],
        [0.2506, 0.4131, 0.3362],
        [0.2543, 0.4173, 0.3284],
        [0.2931, 0.4127, 0.2942],
        [0.2445, 0.4119, 0.3435],
        [0.3531, 0.3649, 0.2820],
        [0.2674, 0.3922, 0.3405],
        [0.2459, 0.4375, 0.3166],
        [0.3117, 0.3904, 0.2979],
        [0.2986, 0.3968, 0.3046],
        [0.2382, 0.3579, 0.4039],
        [0.2578, 0.4053, 0.3369],
        [0.2378, 0.2610, 0.5013],
        [0.4193, 0.3930, 0.1877],
        [0.3227, 0.3932, 0.2841],
        [0.3184, 0.3635, 0.3181],
        [0.2333, 0.4376, 0.3290],
        [0.2547, 0.4580, 0.2873],
        [0.3527, 0.3686, 0.2787],
        [0.4835, 0.3148, 0.2017],
        [0.2740, 0.4182, 0.3078],
        [0.4946, 0.2743, 0.2311],
        [0.4389, 0.3293, 0.2318],
        [0.3278, 0.3581, 0.3141],
        [0.2357, 0.4562, 0.3081],
        [0.2565, 0.4385, 0.3050],
        [0.2595, 0.3919, 0.3486],
        [0.240

In [ ]:
yTest_dev

tensor([0, 1, 0, 0, 2, 1, 0, 1, 1, 0, 1, 0, 2, 1, 1, 0, 0, 0, 0, 1, 1, 2, 2, 0,
        1, 0, 2, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 2, 1, 1, 1, 1, 1,
        0, 0, 2, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 2, 0, 1, 1,
        1, 2, 1, 1, 0, 0, 1, 0, 2])

In [ ]:
yHat_dev

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 0, 1, 1, 1, 1, 1, 0, 1, 0,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 0, 1, 1, 2, 1, 1, 1,
        1, 1, 0, 2, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1,
        2, 1, 1, 1, 0, 2, 0, 0, 1])

In [ ]:
cscoreMLP = []
for i in range(len(yHat_dev)):
    cscoreMLP.append(predProbs_dev[i][yHat_dev[i]])

In [ ]:
cscoreMLP

[tensor(0.4338),
 tensor(0.4465),
 tensor(0.4104),
 tensor(0.4131),
 tensor(0.4173),
 tensor(0.4127),
 tensor(0.4119),
 tensor(0.3649),
 tensor(0.3922),
 tensor(0.4375),
 tensor(0.3904),
 tensor(0.3968),
 tensor(0.4039),
 tensor(0.4053),
 tensor(0.5013),
 tensor(0.4193),
 tensor(0.3932),
 tensor(0.3635),
 tensor(0.4376),
 tensor(0.4580),
 tensor(0.3686),
 tensor(0.4835),
 tensor(0.4182),
 tensor(0.4946),
 tensor(0.4389),
 tensor(0.3581),
 tensor(0.4562),
 tensor(0.4385),
 tensor(0.3919),
 tensor(0.4301),
 tensor(0.3627),
 tensor(0.3782),
 tensor(0.4406),
 tensor(0.3997),
 tensor(0.4178),
 tensor(0.3494),
 tensor(0.3883),
 tensor(0.3848),
 tensor(0.3652),
 tensor(0.4319),
 tensor(0.3950),
 tensor(0.4600),
 tensor(0.4294),
 tensor(0.3999),
 tensor(0.3771),
 tensor(0.4223),
 tensor(0.3852),
 tensor(0.4052),
 tensor(0.3952),
 tensor(0.4042),
 tensor(0.4974),
 tensor(0.3673),
 tensor(0.3983),
 tensor(0.3600),
 tensor(0.3832),
 tensor(0.4238),
 tensor(0.3949),
 tensor(0.4282),
 tensor(0.4130

In [ ]:
oneList = [1]*len(cscoreMLP)

In [ ]:
from sklearn.metrics import mean_squared_error

rms = mean_squared_error(oneList,cscoreMLP, squared=False)

In [ ]:
rms

0.5891768079836841